In [13]:
import pandas as pd

In [15]:
forms_df = pd.read_csv('res_subset_forms.csv')

grouped_forms = forms_df.groupby('unit_id').agg(
    forms_id=('forms_id', 'first'),
    gloss_id=('gloss_id', 'first'),
    gloss=('gloss', 'first'),
    form=('form', 'first'),  
    source=('source', 'first'), 
    all_forms=('form', lambda x: ' '.join(x))  
).reset_index()

sem_df = pd.read_csv("res_subset_sem.csv")

data = pd.merge(
    sem_df,
    grouped_forms.drop_duplicates(subset="unit_id", keep="first"),
    on="unit_id", how="left"
)

In [16]:
def process_entry(row):
    entries = []   
    full_entry = row['full_entry'] 
    if row['gendered'] == 1:
        words = full_entry.split()
        
        start_index = 0
        
        if words[0] in ['шб.', 'шх.', 'ш.']:
            start_index = 1  

        male_form = words[start_index].strip(',')
        female_form = ''
        
        for i in range(len(words)):
            if words[i] == 'ж.':
                if i + 1 < len(words) and words[i + 1] in ['шб.', 'шх.', 'ш.']:
                    female_form = words[i + 2].strip(',') if i + 2 < len(words) else '' 
                else:
                    female_form = words[i + 1].strip(',') if i + 1 < len(words) else '' 

        new_row_male = row.copy()
        new_row_male['full_entry'] = full_entry.replace(f'{male_form}', '').replace(f'{female_form}', '').strip()
        new_row_male['full_entry'] = f"{male_form} " + new_row_male['full_entry']  
        new_row_male['meaning_gender'] = 'm'
        new_row_male['meaning_id'] = f"{row['meaning_id']}" 
        new_row_male['unit_id'] = f"{row['unit_id']}" 
        new_row_male['form'] = row['form'] 
        entries.append(new_row_male)

        if female_form:
            new_row_female = row.copy()
            new_row_female['full_entry'] = full_entry.replace(f'{female_form}', '').replace(f'{male_form}', '').strip()
            new_row_female['full_entry'] = f"{female_form} " + new_row_female['full_entry']  
            new_row_female['meaning_gender'] = 'f'
            new_row_female['meaning_id'] = f"0{row['meaning_id']}" 
            new_row_female['unit_id'] = f"0{row['unit_id']}" 

            all_forms_list = row['all_forms'].split()
            if len(all_forms_list) > 1:
                new_row_female['form'] = all_forms_list[1]  
            else:
                new_row_female['form'] = row['form'] 

            entries.append(new_row_female)

    if not entries:
        entries.append(row)
    
    return entries

result_entries = []
for index, row in data.iterrows():
    result_entries.extend(process_entry(row))
result_df = pd.DataFrame(result_entries)

result_df[result_df['gendered'] == 1]

,meaning_id,unit_id,full_entry,meaning,article_gender,meaning_gender,gendered,comment,suffix,suffix_bVc,...,sicknesses,meanings_concatenated,meaning_with_examples,label_id,forms_id,gloss_id,gloss,form,source,all_forms
37,19653,16758,"ЧЕХТ-ЧИЛАМБ(АК) , ж. зигзагообразный, извилис...","зигзагообразный, извилистый путь, что когда ид...",fm,m,1,NaN,(ak),0,...,0,"зигзагообразный, извилистый путь, что когда ид...","зигзагообразный, извилистый путь, что когда ид...",NaN,25425,20,word,čext-čilambak,2,čext-čilambak čext-čilamb
37,019653,016758,"ЧАХТ-ЧИЛАМБАК , ж. зигзагообразный, извилисты...","зигзагообразный, извилистый путь, что когда ид...",fm,f,1,NaN,(ak),0,...,0,"зигзагообразный, извилистый путь, что когда ид...","зигзагообразный, извилистый путь, что когда ид...",NaN,25425,20,word,čext-čilamb,2,čext-čilambak čext-čilamb
238,5018,3883,"WӮРҶБУЦ шб. , брв. WУРҶБУЦ; ж. шб. , брв. WУРЧ...",волчонок,fm,m,1,animal,buc,1,...,0,волчонок,волчонок,NaN,8357,20,word,wūrǰbuc,2,wūrǰbuc wūrǰbic
238,05018,03883,"WӮРҶБИЦ шб. , брв. WУРҶБУЦ; ж. шб. , брв. WУРЧ...",волчонок,fm,f,1,animal,buc,1,...,0,волчонок,волчонок,NaN,8357,20,word,wūrǰbic,2,wūrǰbuc wūrǰbic
243,11579,9665,НОСОЙИХ̌-БУЦ. ш. ж. НОСОЙИХ̌-БИЦ. б. НОСОЙАХ̌...,"непоседа; неугомонный, беспокойный (парень или...",fm,m,1,abstr human,buc,1,...,0,"непоседа; неугомонный, беспокойный (парень или...","непоседа; неугомонный, беспокойный (парень или...",NaN,16135,20,word,nosoyix̌-buc,2,nosoyix̌-buc nosoyix̌-bic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2944,021239,018140,"ҒАТ-ҒАТ , ж. гогот, шипение (гусей); кряканье...","гогот, шипение (гусей); кряканье (уток); кудах...",fm,f,1,onomatopoeia,NaN,0,...,0,"гогот, шипение (гусей); кряканье (уток); кудах...","гогот, шипение (гусей); кряканье (уток); кудах...",NaN,27220,20,word,ɣat-ɣat,2,ɣut-ɣut ɣat-ɣat
3029,21951,18767,"ҚУ́Т-ҚУ́Т 1, ж. кудахтанье; қу́т-қу́т (қа́т-қ...",кудахтанье,fm,m,1,onomatopoeia,NaN,0,...,0,кудахтанье,кудахтанье,NaN,28015,20,word,qut-qut,2,qut-qut qat-qat
3029,021951,018767,"ҚА́Т-ҚА́Т 1, ж. кудахтанье; қу́т-қу́т (қа́т-қ...",кудахтанье,fm,f,1,onomatopoeia,NaN,0,...,0,кудахтанье,кудахтанье,NaN,28015,20,word,qat-qat,2,qut-qut qat-qat
3095,23274,16754,"ЧÊХТ , ж. , ЧАХТ 1. кривой, кособокий; согнуты...","криво, косо, согнувшись",fm,m,1,adjective,NaN,0,...,0,"кривой, кособокий; согнутый, сутулый; криво, к...","криво, косо, согнувшись; эй, парень, что-то и ...",NaN,29907,20,word,čêxt,2,čêxt čāxt čaxt


In [17]:
result_df.to_csv('data.csv')